In [12]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
path = 'dataset/'
os.listdir(path)

['icml_face_data.csv']

In [19]:
data = pd.read_csv(path+'icml_face_data.csv')
data.columns = ['emotion', 'usage', ' pixels']
data.head()

,emotion,usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [128]:
def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48, 3))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = np.repeat(np.expand_dims(image, axis=2), 3, axis=2)
        
    return image_array, image_label

In [129]:
data['usage'].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: usage, dtype: int64

In [130]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [131]:
# datasets for development 
train_data = data[data['usage']=='Training'][:1000]
val_data = data[data['usage']=='PrivateTest'][:100]
test_data = data[data['usage']=='PublicTest'][:100]

train_img, train_img_label = prepare_data(train_data)
val_img, val_img_label = prepare_data(val_data)
test_img, test_img_label = prepare_data(test_data)

train_images = train_img.reshape((train_img.shape[0], 48, 48, 3))
train_images = train_images.astype('float32')/255
val_images = val_img.reshape((val_img.shape[0], 48, 48, 3))
val_images = val_images.astype('float32')/255
test_images = test_img.reshape((test_img.shape[0], 48, 48, 3))
test_images = test_images.astype('float32')/255

In [132]:
train_labels = to_categorical(train_img_label)
val_labels = to_categorical(val_img_label)
test_labels = to_categorical(test_img_label)

In [133]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input

idg = ImageDataGenerator(rescale=1./255., preprocessing_function=preprocess_input)

train_dataset = idg.flow(
    train_images,
    train_labels,
    batch_size=64,
    shuffle=True
)

val_dataset = idg.flow(
    val_images,
    val_labels,
    batch_size=64,
    shuffle=True
)

test_dataset = idg.flow(
    test_images,
    test_labels,
    batch_size=64
)

print(train_dataset.n)
print(val_dataset.n)
print(test_dataset.n)

1000
100
100


In [134]:
class_weight = dict(zip(range(0, 7), (((data[data['usage']=='Training']['emotion'].value_counts()).sort_index())/len(data[data['usage']=='Training']['emotion'])).tolist()))
class_weight

{0: 0.1391549688251071,
 1: 0.01518687519593159,
 2: 0.14270786164617366,
 3: 0.2513149186666202,
 4: 0.16823992476226968,
 5: 0.11045316799609878,
 6: 0.17294228290779895}

In [141]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input

vgg_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape = (48,48,3)
)

In [142]:
vgg_model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [148]:
history = vgg_model.fit(
    train_images,
    train_labels,
    validation_data=(val_images, val_labels),
    class_weight = class_weight,
    epochs=30,
    batch_size=16,
#     callbacks=EarlyStopping(monitor='val_loss')
)

Epoch 1/30


ValueError: in user code:

    /home/dilip/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/dilip/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/dilip/.local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/dilip/.local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/dilip/.local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/dilip/.local/lib/python3.7/site-packages/tensorflow/python/keras/engine/traini